In [1]:
# Importações necessárias
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Dot
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
from pathlib import Path  

**Algumas configurações necessárias**

In [2]:
plt.style.use('ggplot')  
plt.rcParams.update({
    'font.size': 14,
    'figure.figsize': (12, 8),  
    'axes.titlesize': 18,
    'axes.labelsize': 14,
    'legend.fontsize': 12,
})

**Geração de dados sintéticos**

In [3]:
num_users = 100
num_products = 50
embedding_dim = 5

# Gerar embeddings aleatórios para usuários e produtos
user_embeddings = np.random.randn(num_users, embedding_dim)
product_embeddings = np.random.randn(num_products, embedding_dim)

# Gerar interações (notas de 1 a 5)
np.random.seed(42)
ratings = []
user_ids = []
product_ids = []

for _ in range(2000):
    user = np.random.randint(0, num_users)
    product = np.random.randint(0, num_products)
    # Nota baseada na similaridade dos embeddings + ruído
    rating = np.dot(user_embeddings[user], product_embeddings[product]) + np.random.normal(0, 0.5)
    ratings.append(np.clip(rating, 1, 5))
    user_ids.append(user)
    product_ids.append(product)

ratings = np.array(ratings)
user_ids = np.array(user_ids)
product_ids = np.array(product_ids)

**Construção do modelo de recomendação**

In [4]:
input_user = Input(shape=(1,))
input_product = Input(shape=(1,))

# Camadas de embedding
user_embedding = Embedding(num_users, embedding_dim)(input_user)
product_embedding = Embedding(num_products, embedding_dim)(input_product)

# Achatar embeddings
user_vec = Flatten()(user_embedding)
product_vec = Flatten()(product_embedding)

# Produto ponto-a-ponto + camada densa
prod = Dot(axes=1)([user_vec, product_vec])
output = Dense(1, activation='linear')(prod)

model = Model(inputs=[input_user, input_product], outputs=output)
model.compile(optimizer='adam', loss='mse')

**Treinamento do modelo com animação**

In [5]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 14), gridspec_kw={'hspace': 0.5})

# Armazenar métricas para animação
losses = []
val_losses = []
epochs = 15
batch_size = 64

# Função de atualização com visualização aprimorada
def update(frame):
    ax1.clear()
    ax2.clear()
    
    # Treinar por 1 epoch
    history = model.fit(
        [user_ids, product_ids],
        ratings,
        epochs=1,
        batch_size=batch_size,
        verbose=0,
        validation_split=0.2
    )
    
    losses.append(history.history['loss'][0])
    val_losses.append(history.history['val_loss'][0])
    
    # PLOT 1: PERDA DE TREINAMENTO E VALIDAÇÃO
    ax1.plot(range(len(losses)), losses, 'o-', color='#1f77b4', linewidth=2, markersize=6, label='Treinamento')
    ax1.plot(range(len(val_losses)), val_losses, 's--', color='#ff7f0e', linewidth=2, markersize=6, label='Validação')
    ax1.set_title('Progresso do Treinamento - Perda MSE', fontsize=16, fontweight='bold')
    ax1.set_xlabel('Épocas', fontsize=14)
    ax1.set_ylabel('Perda (MSE)', fontsize=14)
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, max(max(losses), max(val_losses)) * 1.1)
    
    # PLOT 2: PREVISÕES PARA EXEMPLOS ESPECÍFICOS
    test_user = np.array([0, 1, 2, 3, 4])
    test_product = np.array([0, 1, 2, 3, 4])
    predictions = model.predict([test_user, test_product]).flatten()
    
    bars = ax2.bar(range(len(predictions)), predictions, color=['#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2'])
    ax2.set_title('Previsões para Usuários Selecionados', fontsize=16, fontweight='bold')
    ax2.set_xlabel('Usuário/Produto', fontsize=14)
    ax2.set_ylabel('Nota Prevista', fontsize=14)
    ax2.set_xticks(range(len(test_user)))
    ax2.set_xticklabels([f'U{u}/P{p}' for u, p in zip(test_user, test_product)])
    ax2.set_ylim(1, 5)
    ax2.grid(axis='y', alpha=0.3)
    
    # Adicionar valores nas barras
    for bar in bars:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.05,
                f'{height:.2f}', ha='center', va='bottom', fontsize=12)
    
    # Informações adicionais
    ax1.annotate(f'Época {frame+1}/{epochs}\nÚltima Perda: {losses[-1]:.4f}',
                 xy=(0.02, 0.98), xycoords='axes fraction',
                 fontsize=12, bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray"),
                 verticalalignment='top')

# Criar animação
ani = animation.FuncAnimation(fig, update, frames=epochs, interval=800, repeat=False)

plt.close()

**Salvando o modelo gerado**

In [7]:
# Definir caminho absoluto para a pasta media na raiz do projeto
current_dir = os.getcwd()
project_root = Path(current_dir).parent  

# Definindo media_path 
media_path = project_root / 'media'

# Criar pasta media se não existir
media_path.mkdir(exist_ok=True)

# Salvar animação
gif_path = media_path / 'recomendacao_animacao_impactante.gif'
ani.save(gif_path, writer='pillow', fps=2)

plt.close()
print(f"Animação salva como '{gif_path}'!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Animação salva como 'C:\Users\flavi\Documents\GitHub\Projeto_Recomendacao_Animacao\media\recomendacao_animacao_impactante.gif'!
